In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In this notebook we merged the country name, religion, area, population, gdp per capita and gov_type_num features. We used ISO alpha-2 code to do the merging. 

In [2]:
# Reading excel and selecting most recent data
rel_df = pd.read_excel('World Religion Dataset - National Religion Dataset.xlsx')
rel_df = rel_df.loc[rel_df['YEAR'] == '2010']

# Renaming religion columns
new_names = {'CHGENPCT': 'Christianism', 'JDGENPCT': 'Judaism', 'ISGENPCT': 'Islam', 'BUGENPCT': 'Buddhism',
             'ZOGENPCT': 'Zoroastrian', 'HIGENPCT': 'Hindu', 'SIGENPCT': 'Sikh', 'SHGENPCT': 'Shinto', 
             'BAGENPCT': "Baha'i", 'TAGENPCT': 'Taoism', 'JAGENPCT': 'Jain', 'COGENPCT': 'Confucianism', 
             'SYGENPCT': 'Syncretic religions', 'ANGENPCT': 'Animist religions', 'NORELPCT': 'Non-religious', 
             'OTGENPCT': 'Other religions'}

religion_cols = list(new_names.values())
rel_df.rename(columns=new_names, inplace=True)

# Selecing useful columns
cols = ['ISO3'] + religion_cols
rel_df = rel_df[cols]

# Reseting index
rel_df.reset_index(drop=True, inplace=True)

rel_df.head()

,ISO3,Christianism,Judaism,Islam,Buddhism,Zoroastrian,Hindu,Sikh,Shinto,Baha'i,Taoism,Jain,Confucianism,Syncretic religions,Animist religions,Non-religious,Other religions
0,USA,0.7454,0.018999,0.008999,0.010899,0.005299,0.005699,0.001299,0.0005,0.0015,0.000000,0.000300,0.000300,0.002599,0.005699,0.190000,0.0025
1,CAN,0.7661,0.009899,0.019399,0.019399,0.000200,0.007999,0.007999,0.0000,0.0005,0.000099,0.000099,0.000099,0.000800,0.002100,0.164300,0.0010
2,BHS,0.9660,0.001000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000300,0.000000,0.000000,0.000000,0.003200,0.028999,0.0005
3,CUB,0.6589,0.000099,0.000700,0.000000,0.000000,0.002200,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.500000,0.000000,0.131500,0.0000
4,HTI,0.8200,0.000000,0.000200,0.000000,0.000000,0.000000,0.000000,0.0000,0.0009,0.000000,0.000000,0.000000,0.450000,0.000000,0.100000,0.0000


As we said, we will merge on ISO2, but we have ISO3 in rel_df. In order to map ISO3 to ISO2 codes we will use the data.pickle dataframe which contains both ISO codes.

In [3]:
# Reading dataframe used to map country codes
data = pd.read_pickle('data.pickle')
data.reset_index(inplace=True)

# Merging the two dataframes and selecting useful columns
final_rel_df = pd.merge(data, rel_df, on='ISO3')
final_rel_df = final_rel_df[['ISO2', 'name'] + religion_cols]
final_rel_df.set_index('ISO2', inplace=True)
final_rel_df.head(2)

,name,Christianism,Judaism,Islam,Buddhism,Zoroastrian,Hindu,Sikh,Shinto,Baha'i,Taoism,Jain,Confucianism,Syncretic religions,Animist religions,Non-religious,Other religions
ISO2,,,,,,,,,,,,,,,,,
AF,Afghanistan,0.0003,0.0,0.995600,0.000099,0.000099,0.0003,0.000099,0.0,0.000099,0.0,0.0,0.0,0.0,0.000099,0.002000,0.0014
AO,Angola,0.8912,0.0,0.010399,0.000099,0.000000,0.0000,0.000000,0.0,0.000099,0.0,0.0,0.0,0.0,0.075899,0.017899,0.0044


Now we want to reformat the names of the columns of the dataframe so that we end with all religion column names under the same column name: religion.

In [4]:
# Creating the tuple array for the names of the columns
religion_array = list()
for rel in final_rel_df.columns[1:]:
    religion_array.append(('religion', rel))

# Assigning the hierarchical column names
final_rel_df.columns = pd.MultiIndex.from_tuples([('name','')] + religion_array)

In [5]:
final_rel_df.head()

name     religion                              \
                           Christianism Judaism     Islam  Buddhism   
ISO2                                                                  
AF             Afghanistan       0.0003     0.0  0.995600  0.000099   
AO                  Angola       0.8912     0.0  0.010399  0.000099   
AL                 Albania       0.2144     0.0  0.630000  0.000000   
AD                 Andorra       0.9070     0.0  0.008999  0.000000   
AE    United Arab Emirates       0.0714     0.0  0.674800  0.003500   

                                                                              \
     Zoroastrian   Hindu      Sikh Shinto    Baha'i Taoism Jain Confucianism   
ISO2                                                                           
AF      0.000099  0.0003  0.000099    0.0  0.000099    0.0  0.0          0.0   
AO      0.000000  0.0000  0.000000    0.0  0.000099    0.0  0.0          0.0   
AL      0.000000  0.0000  0.000000    0.0  0.002200    0.0  0.0          0.0   
AD      0.000000  0.0035  0.000000    0.0  0.000000    0.0  0.0          0.0   
AE      0.000000  0.2225  0.000000    0.0  0.009999    0.0  0.0          0.0   

                                                                          
     Syncretic religions Animist religions Non-religious Other religions  
ISO2                                                                      
AF                   0.0          0.000099      0.002000          0.0014  
AO                   0.0          0.075899      0.017899          0.0044  
AL                   0.0          0.000000      0.150700          0.0027  
AD                   0.0          0.000000      0.079600          0.0009  
AE                   0.0          0.000000      0.013599          0.0041

In [6]:
# We set the index to ISO2
data.set_index('ISO2', inplace=True)

In [7]:
# Reading and selecting useful data
area_POP_gdp_df = data[['area', 'POP', '2016_gdp_capita']]
area_POP_gdp_df.head()

,area,POP,2016_gdp_capita
ISO2,,,
AW,180.0,NaN,NaN
AF,652230.0,27000000.0,561.778746
AO,1246700.0,19114176.0,3110.808183
AI,91.0,NaN,NaN
AX,1580.0,NaN,NaN


In [8]:
# Selecting gov type dataframe
gov_type_df = data[['gov_type_num']]
gov_type_df.head()

,gov_type_num
ISO2,
AW,-1.0
AF,1.0
AO,1.0
AI,-1.0
AX,0.0


Finally, we merge all the dataframes by the index ISO2.

In [9]:
# Merging all three dataframes
df = pd.concat([final_rel_df, area_POP_gdp_df], axis=1)
df = pd.concat([df, gov_type_df], axis=1)

# Renaming columns with hierarchical names
df.columns = pd.MultiIndex.from_tuples([('name', '')] + religion_array + \
                                       [('area', ''), ('POP', ''), ('2016_gdp_capita', ''), ('gov_type_num', '')])
df.head()

name     religion                                          \
                         Christianism Judaism     Islam  Buddhism Zoroastrian   
AD               Andorra       0.9070     0.0  0.008999  0.000000    0.000000   
AE  United Arab Emirates       0.0714     0.0  0.674800  0.003500    0.000000   
AF           Afghanistan       0.0003     0.0  0.995600  0.000099    0.000099   
AG                   NaN          NaN     NaN       NaN       NaN         NaN   
AI                   NaN          NaN     NaN       NaN       NaN         NaN   

                                          ...                         \
     Hindu      Sikh Shinto    Baha'i     ...      Jain Confucianism   
AD  0.0035  0.000000    0.0  0.000000     ...       0.0          0.0   
AE  0.2225  0.000000    0.0  0.009999     ...       0.0          0.0   
AF  0.0003  0.000099    0.0  0.000099     ...       0.0          0.0   
AG     NaN       NaN    NaN       NaN     ...       NaN          NaN   
AI     NaN       NaN    NaN       NaN     ...       NaN          NaN   

                                                                        \
   Syncretic religions Animist religions Non-religious Other religions   
AD                 0.0          0.000000      0.079600          0.0009   
AE                 0.0          0.000000      0.013599          0.0041   
AF                 0.0          0.000099      0.002000          0.0014   
AG                 NaN               NaN           NaN             NaN   
AI                 NaN               NaN           NaN             NaN   

        area         POP 2016_gdp_capita gov_type_num  
                                                       
AD     468.0     85500.0             NaN         -1.0  
AE   83600.0   6236650.0    37622.207458         -1.0  
AF  652230.0  27000000.0      561.778746          1.0  
AG     442.0         NaN    14353.378814         -1.0  
AI      91.0         NaN             NaN         -1.0  

[5 rows x 21 columns]

After merging, we get quite a few NaN values because the dataframes don't have exactly the same countries so we want to check the order of magnitude of the NaN values.

In [80]:
# Check for number of rows containing 
df[df.isnull().any(axis=1)]df.hist(figsize=(20, 100), layout=(20,3));
plt.tight_layout()

name     religion                                           \
                     Christianism   Judaism     Islam Buddhism Zoroastrian   
AD           Andorra     0.907000  0.000000  0.008999   0.0000         0.0   
AG               NaN          NaN       NaN       NaN      NaN         NaN   
AI               NaN          NaN       NaN       NaN      NaN         NaN   
AQ               NaN          NaN       NaN       NaN      NaN         NaN   
AS               NaN          NaN       NaN       NaN      NaN         NaN   
AW               NaN          NaN       NaN       NaN      NaN         NaN   
AX               NaN          NaN       NaN       NaN      NaN         NaN   
BL               NaN          NaN       NaN       NaN      NaN         NaN   
BM               NaN          NaN       NaN       NaN      NaN         NaN   
BV               NaN          NaN       NaN       NaN      NaN         NaN   
CC               NaN          NaN       NaN       NaN      NaN         NaN   
CK               NaN          NaN       NaN       NaN      NaN         NaN   
CU              Cuba     0.658900  0.000099  0.000700   0.0000         0.0   
CW               NaN          NaN       NaN       NaN      NaN         NaN   
CX               NaN          NaN       NaN       NaN      NaN         NaN   
DJ          Djibouti     0.013899  0.000000  0.975000   0.0000         0.0   
EH               NaN          NaN       NaN       NaN      NaN         NaN   
ER           Eritrea     0.575000  0.000000  0.401200   0.0000         0.0   
FK               NaN          NaN       NaN       NaN      NaN         NaN   
FO               NaN          NaN       NaN       NaN      NaN         NaN   
GF               NaN          NaN       NaN       NaN      NaN         NaN   
GG               NaN          NaN       NaN       NaN      NaN         NaN   
GI               NaN          NaN       NaN       NaN      NaN         NaN   
GL               NaN          NaN       NaN       NaN      NaN         NaN   
GP               NaN          NaN       NaN       NaN      NaN         NaN   
GS               NaN          NaN       NaN       NaN      NaN         NaN   
GU               NaN          NaN       NaN       NaN      NaN         NaN   
HK               NaN          NaN       NaN       NaN      NaN         NaN   
HM               NaN          NaN       NaN       NaN      NaN         NaN   
IM               NaN          NaN       NaN       NaN      NaN         NaN   
..               ...          ...       ...       ...      ...         ...   
MS               NaN          NaN       NaN       NaN      NaN         NaN   
NC               NaN          NaN       NaN       NaN      NaN         NaN   
NF               NaN          NaN       NaN       NaN      NaN         NaN   
NU               NaN          NaN       NaN       NaN      NaN         NaN   
PF               NaN          NaN       NaN       NaN      NaN         NaN   
PG  Papua New Guinea     0.960000  0.000099  0.000300   0.0004         0.0   
PM               NaN          NaN       NaN       NaN      NaN         NaN   
PN               NaN          NaN       NaN       NaN      NaN         NaN   
PR               NaN          NaN       NaN       NaN      NaN         NaN   
PS               NaN          NaN       NaN       NaN      NaN         NaN   
RE               NaN          NaN       NaN       NaN      NaN         NaN   
SJ               NaN          NaN       NaN       NaN      NaN         NaN   
SM        San Marino     0.863100  0.000000  0.000200   0.0000         0.0   
SS               NaN          NaN       NaN       NaN      NaN         NaN   
SX               NaN          NaN       NaN       NaN      NaN         NaN   
SY             Syria     0.075200  0.000000  0.902000   0.0000         0.0   
TC               NaN          NaN       NaN       NaN      NaN         NaN   
TF               NaN          NaN       NaN       NaN      NaN         NaN   
TK               NaN          NaN     

In [86]:
df.to_pickle('6_feature_df.pickle')

# Delete all following cells

In [97]:
df.loc[df['2016_gdp_capita'].isnull(), [('2016_gdp_capita',''), ('POP','')]]

,2016_gdp_capita,POP
,,
AD,NaN,85500.0
AI,NaN,NaN
AQ,NaN,NaN
AS,NaN,NaN
AW,NaN,NaN
AX,NaN,NaN
BL,NaN,NaN
BM,NaN,NaN
BV,NaN,NaN


In [99]:
data = pd.read_pickle('data.pickle')
data.gov_type

name
Aruba                                    parliamentary democracy
Afghanistan                                presidential republic
Angola                                     presidential republic
Anguilla                                 parliamentary democracy
Åland Islands                                            unknown
Albania                                   parliamentary republic
Andorra                                  parliamentary democracy
United Arab Emirates                    federation of monarchies
Argentina                                  presidential republic
Armenia                                    presidential republic
American Samoa                            presidential democracy
Antarctica                                               unknown
French Southern and Antarctic Lands                      unknown
Antigua and Barbuda                      parliamentary democracy
Australia                                parliamentary democracy
Austria             